### Module

In [2]:
import os
import sys
from dotenv import load_dotenv
import textwrap
from IPython.display import display
from IPython.display import Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


### SQL 서버 연동

In [17]:
import mysql.connector
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# MySQL 연결 설정
mysql_config = {
    'user': 'root',
    'password': 'admin',
    'host': '175.45.205.178',
    'port': '3306',
    'database': 'vectorstoreDB'
}

# Create a connection to the MySQL database
try:
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    print("Successfully connected to the database")

    # Your database operations here

except mysql.connector.Error as err:
    print(f"Error: {err}")

# finally:
#     if connection.is_connected():
#         cursor.close()
#         connection.close()
#         print("MySQL connection is closed")


Successfully connected to the database


In [18]:

# 데이터베이스와 테이블 생성 (필요한 경우)

cursor = connection.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS your_database")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS embeddings (
        id INT AUTO_INCREMENT PRIMARY KEY,
        document_id VARCHAR(255) NOT NULL,
        vector BLOB NOT NULL,
        metadata JSON
    )
""")


In [26]:
from chromadb import PersistentClient
from chromadb import HttpClient
from chromadb.config import Settings
from langchain_community.embeddings import OpenAIEmbeddings
import mysql.connector

# Chroma 벡터 데이터베이스 로드
vector_db_path = "/home/user/khtml-ai-llm-tt/vectordb"
api_key = OPENAI_API_KEY

class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        # input은 텍스트 리스트여야 하며, 이를 임베딩합니다.
        return super().__call__(input)

# 맞춤형 임베딩 함수 사용
embeddings = CustomOpenAIEmbeddings(api_key=api_key)

client = PersistentClient(path=vector_db_path)
# 컬렉션 가져오거나, 없으면 생성
collection_name = "vectordb1"
collection = client.get_or_create_collection(name=collection_name, embedding_function=embeddings)
print(collection)
# 이후 컬렉션을 사용할 수 있습니다.


Collection(id=74acc0cb-9a0a-498c-a537-08ae8ef5dbbd, name=vectordb1)


In [27]:
# 컬렉션의 모든 항목 가져오기
results = collection.get()
print(results)

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': [], 'uris': None, 'data': None, 'included': ['metadatas', 'documents']}


### pesistent 방식 저장

In [32]:
# PDF 파일이 저장된 디렉토리 경로
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
pdf_directory = "/home/user/khtml-ai-llm-tt/data/academic resources"

def load_pdf_files(self, pdf_directory):
        documents = []
        print("Start to load PDFs.")
        pdf_files = glob(os.path.join(file_directory, '*.pdf'))
        for pdf_file in pdf_files:
            loader = PyPDFLoader(pdf_file)
            pdf_documents = loader.load()
            documents.extend(pdf_documents)
        print(f"Loaded {len(documents)} PDF documents.")
        return documents

def split_documents(self, documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_db(self, chunks):
        if not chunks:
            print("No chunks to save to VectorDB.")
            return None
        print("Save VectorDB")
        vectordb = Chroma.from_documents(documents=chunks, embedding=self.embeddings, persist_directory=self.vector_db_path)
        vectordb.persist()
        return vectordb.as_retriever()
 


print("모든 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")       
# 데이터 로드 및 확인
loaded_collection = client.get_collection(name=collection_name, embedding_function=embeddings)
print(f"컬렉션 '{collection_name}'의 항목 수: {loaded_collection.count()}")

모든 PDF 파일이 벡터 데이터베이스에 저장되었습니다.
컬렉션 'vectordb1'의 항목 수: 0


In [57]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb

# OpenAI API 키 설정
api_key = OPENAI_API_KEY

# 사용자 정의 임베딩 클래스 정의
class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        # 입력은 텍스트 리스트여야 합니다. 이를 임베딩합니다.
        return super().__call__(input)

# 사용자 정의 임베딩 함수 사용
embeddings = CustomOpenAIEmbeddings(api_key=api_key)

# Chroma DB 설정
vector_db_path = "/home/user/khtml-ai-llm/vectordb"

# PersistentClient 초기화
client = chromadb.PersistentClient(path=vector_db_path)

# 컬렉션 생성 또는 가져오기
collection_name = "vectordb1"
collection = client.get_or_create_collection(name=collection_name, embedding_function=embeddings)

# PDF 파일이 저장된 디렉토리 경로
pdf_directory = "/home/user/khtml-ai-llm-tt/data/academic resources"

def load_pdf_files(pdf_directory):
    documents = []
    print("Start to load PDFs.")
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        print(f"Loaded {len(pdf_documents)} documents from {pdf_file}")
        documents.extend(pdf_documents)
    print(f"Loaded {len(documents)} PDF documents.")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_db(chunks, embeddings, vector_db_path):
    if not chunks:
        print("No chunks to save to VectorDB.")
        return None
    print("Save VectorDB")
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=vector_db_path)
    vectordb.add_documents(documents=chunks)
    vectordb.persist()
    return vectordb.as_retriever()

# PDF 파일 로드
documents = load_pdf_files(pdf_directory)

# 문서 분할
chunks = split_documents(documents)

# 데이터베이스에 저장
retriever = save_db(chunks, embeddings, vector_db_path)

print("모든 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")


Start to load PDFs.
Found 31 PDF files.
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_결석조치.pdf
Loaded 4 documents 

In [63]:

# 데이터 로드 및 확인
loaded_collection = client.get_collection(name=collection_name, embedding_function=embeddings)
print(f"컬렉션 '{collection_name}'의 항목 수: {loaded_collection.count()}")


컬렉션 'vectordb1'의 항목 수: 0


#### gemini editing code

In [65]:

import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import chromadb

# OpenAI API 키 설정
api_key = OPENAI_API_KEY

# 사용자 정의 임베딩 클래스 정의
class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        # 입력은 텍스트 리스트여야 합니다. 이를 임베딩합니다.
        return super().__call__(input)

# 사용자 정의 임베딩 함수 사용
embeddings = CustomOpenAIEmbeddings(api_key=api_key)

# Chroma DB 설정
vector_db_path = "/home/user/khtml-ai-llm/vectordb"

# PersistentClient 초기화
client = chromadb.PersistentClient(path=vector_db_path)

# 컬렉션 생성 또는 가져오기
collection_name = "vectordb1"
collection = client.get_or_create_collection(name=collection_name, embedding_function=embeddings)

# PDF 파일이 저장된 디렉토리 경로
pdf_directory = "/home/user/khtml-ai-llm-tt/data/academic resources"

def load_pdf_files(pdf_directory):
    documents = []
    print("Start to load PDFs.")
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        print(f"Loaded {len(pdf_documents)} documents from {pdf_file}")
        documents.extend(pdf_documents)
    print(f"Loaded {len(documents)} PDF documents.")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_to_db(chunks, collection):
    if not chunks:
        print("No chunks to save to VectorDB.")
        return
    print("Save VectorDB")
    
    # 청크를 배치로 나누어 추가 (메모리 문제 방지)
    batch_size = 100
    for i in range(0, len(chunks), batch_size):
        collection.add(
            documents=[c.page_content for c in chunks[i:i+batch_size]],
            metadatas=[{"source": c.metadata["source"]} for c in chunks[i:i+batch_size]],
            ids=[str(uuid.uuid4()) for _ in chunks[i:i+batch_size]]
        )

def load_from_db(collection):
    print("Load VectorDB")
    return collection

# PDF 파일 로드
documents = load_pdf_files(pdf_directory)

# 문서 분할
chunks = split_documents(documents)

# 데이터베이스에 저장
save_to_db(chunks, collection)

print("모든 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")

# 데이터베이스에서 로드
db = load_from_db(collection)

# db를 사용하여 질문에 답변하거나 다른 작업 수행


Start to load PDFs.
Found 31 PDF files.
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_결석조치.pdf
Loaded 4 documents 

AttributeError: 'super' object has no attribute '__call__'

In [ ]:

# 데이터 로드 및 확인
loaded_collection = client.get_collection(name=collection_name, embedding_function=embeddings)
print(f"컬렉션 '{collection_name}'의 항목 수: {loaded_collection.count()}")


#### sample code

In [61]:
# Suggested code may be subject to a license. Learn more: ~LicenseLog:4098388073.
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
import glob
import uuid

COLLECTION_NAME = 'obsidian_md_db'
# Persistent Chroma Client 시작
persistent_client = chromadb.PersistentClient()
# 임베딩 함수 설정 (Chroma의 기본 임베딩 함수)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# 이미 COLLECTION_NAME이라는 이름의 컬렉션이 있는지 확인
collections = persistent_client.list_collections()
is_collection_exist = any(collection.name == COLLECTION_NAME for collection in collections)

if is_collection_exist:
    # 기존에 생성된 collection이 존재
    collection = persistent_client.get_collection(COLLECTION_NAME)
else:
    # 기존에 생성된 collection이 존재하지 않음
    # 새로운 collection을 생성
    collection = persistent_client.get_or_create_collection(COLLECTION_NAME)

    # data 폴더에 있는 모든 .md 파일의 경로를 가져옴
    md_files = glob.glob('data/*.md')

    # 각 파일을 읽어서 collection에 추가합
    # chunk_size = 1000, chunk_overlap  = 50 으로 분할하여 embedding을 수행
    for md_file in md_files:
        loader = TextLoader(md_file)
        docs = loader.load()
        tokenized_docs = get_tokenized_text(docs[0].page_content)
        for doc in tokenized_docs:
            collection.add(ids=[str(uuid.uuid1())], metadatas=docs[0].metadata, documents=doc.page_content)


/home/user/khtml-ai-llm-tt/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [62]:
query = "ChatGPT 음성 영어회화 설정방법"

# similarity search 와 유사한 결과
results = collection.query(
    query_texts=[query],
    n_results=5,    # 검색 결과의 개수
)
# 검색된 문서들의 내용들을 출력
print(results['documents'])
# 검색된 문서들의 metadata들을 출력
# 문서의 경로, 파일명, 기타 정보들을 출력할 수 있음
print(results['metadatas'])
# 검색된 문서들을 바탕으로 ChaGPT에 질문
result = process_query_llm(query, results['documents'])
print(result)

# 특정 문서에 있는 내용에서만 검색
results = collection.query(
    query_texts=[query],
    n_results=5,
    where={
        "source": {
            "$eq": "data/AI Agent 만들기.md"
        }
    }
)
result = process_query_llm(query, results['documents'])
print(result)


/home/user/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 10.4MiB/s]


[[]]
[[]]


NameError: name 'process_query_llm' is not defined

### persist만 이용

In [3]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb

# OpenAI API 키 설정
api_key = OPENAI_API_KEY

# 사용자 정의 임베딩 클래스 정의
class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        # 입력은 텍스트 리스트여야 합니다. 이를 임베딩합니다.
        return super().__call__(input)

# 사용자 정의 임베딩 함수 사용
embeddings = CustomOpenAIEmbeddings(api_key=api_key)

# Chroma DB 설정
vector_db_path = "/home/user/khtml-ai-llm/vectordb"

# PersistentClient 초기화
client = chromadb.PersistentClient(path=vector_db_path)

# 컬렉션 생성 또는 가져오기
collection_name = "vectordb1"
collection = client.get_or_create_collection(name=collection_name, embedding_function=embeddings)

# PDF 파일이 저장된 디렉토리 경로
pdf_directory = "/home/user/khtml-ai-llm-tt/data/academic resources"

def load_pdf_files(pdf_directory):
    documents = []
    print("Start to load PDFs.")
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        print(f"Loaded {len(pdf_documents)} documents from {pdf_file}")
        documents.extend(pdf_documents)
    print(f"Loaded {len(documents)} PDF documents.")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_db(chunks, embeddings, vector_db_path):
    if not chunks:
        print("No chunks to save to VectorDB.")
        return None
    print("Save VectorDB")
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=vector_db_path)
    vectordb.add_documents(documents=chunks)
    vectordb.persist()
    return vectordb.as_retriever()

# PDF 파일 로드
documents = load_pdf_files(pdf_directory)

# 문서 분할
chunks = split_documents(documents)

# 데이터베이스에 저장
retriever = save_db(chunks, embeddings, vector_db_path)

print("모든 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")


Start to load PDFs.
Found 31 PDF files.
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/다전공_연계전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/전공_전공.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/학적_제적.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loaded 2 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_강의평가.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loaded 3 documents from /home/user/khtml-ai-llm-tt/data/academic resources/수강_학사구조개편 관련 학사제도 안내.pdf
Loading PDF file: /home/user/khtml-ai-llm-tt/data/academic resources/수강_결석조치.pdf
Loaded 4 documents 

/home/user/khtml-ai-llm-tt/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [9]:
vectordb = Chroma(persist_directory=vector_db_path, 
                  embedding_function=embeddings)
retriever = vectordb.as_retriever()
            

### prompting

In [10]:
KNU_INFO=f"""
    공과대학 은 3학부로 인공지능융합공학부, 부동산건설학부, ICT융합공학부로 이루어져 있어
    
    [학부에 따른 전공]
    인공지능융합공학부 : 인공지능전공, 데이터사이언스전공, 산업공학전공
    부동산건설학부 : 부동산학전공, 스마트도시공학전공, 건축공학전공
    ICT융합공학부 : 소프트웨어전공, 가상현실전공, 전자공학전공

    주전공에서 '전기'는 '전공 기초', '전선'은 '전공 선택'이야. 
    복수전공은 '복기' 혹은 '복수' 라고 된 것이 있어.
"""
OUTPUT_FORMAT=f"""
### 학사 안내 \\
#### 전공·다전공 or 수강 or 학적 에 대한 답변 \\
>  000은 ...하여 ...인 것을 말합니다. \n \\
> ...과 같은 특징을 지니고 있으며, ...에서는 ...의 역할을 합니다. \n \\
> 000에 대해서 주의할 점은 ... 이며 ...입니다. \n \\

### 추천 행동
|행동 | 이유 | 설명 |
|---|---| ---|
|... | ... | ... |
|... | ... | ... |
|... | ... | ... |

### 추가정보
https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do
"""
SYS_PROMPT = f"""
    [역할]
    너는 강남대학교 공과대학 학생들의 학사안내와 관련된 질문에 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    주어진 정보를 반드시 충분히 이해하고 여기에서 설명해야해.
    [해설 정보]
    {KNU_INFO}는 주어진 정보에 대한 해설 정보야.
    학사 안내는 '전공·다전공', '수강' '학적'에 관련된 정보들이 존재해.
    아래의 [] 안에 있는 주제의 관련내용() 안의 키워드를 중심으로 파일을 찾아줘.
   
    [전공·다전공]
    - 전공 관련 내용(다전공, 심화전공, 전공, 전공신청 및 변경, 강남인증제 등)
    - 다전공 관련 내용(다전공, 마이크로전공, 복수-부전공, 연계전공, 융합설계전공, 융합전공, 자율설계전공 등)
    [수강]
    - 수강 관련 내용(강의계획서, 강의평가, 결석조치, 계절수업, 수강신청, 수업피드백시스템, 수업FAQ, 시험및 성적, 인정학점, 재수강, 학사경고, 학사구조개편 관련 학사제도 안내 등)
    [학적]
    - 학적 관련 내용(강남인증제, 자퇴, 재입학, 전부전과, 제적, 학.석사연계과정,학적부정정 등)

    [답변 내용]
    사용자의 답변에 따라서 전공·다전공 or 수강 or 학적 에 대한 답변중 하나의 답변을 정해서 해줘야 해.
    그리고 마지막에 챗봇이 생각하는 가장 사용자에게 적절할 추천 행동을 출력해줘.
    db와 너에게 주어진 정보에 충실하게 대답해야해.


아래 출력 예시를 참고해줘.markdown 형식을 충실히 따라줘.
{OUTPUT_FORMAT}
이 글과 이 아래 부터는 출력 예시가 아니야.
"""

INPUT_PROMPT = f"""
위의 질문에 충실하게 대답해줘. 잘하면 10달러 줄게. \\
"""

### Model

In [11]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
template = SYS_PROMPT + '''
1.강남대학교 AI로드맵을 위한 참고 자료야 :{context}
2.사용자 입력 메세지에 잘 따라줘야해 : {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

input_message =  """
자퇴와 계절수업에 대해서 각각 알려줘
"""+INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))

> ### 학사 안내 
> #### 학적에 대한 답변 
> > **자퇴**는 학생이 자발적으로 학교를 그만두는 것을 말합니다. 자퇴를 신청하려면 학사관리팀에 자퇴 신청서를 제출해야 하며, 자퇴가 승인되면 학적이 말소됩니다. 자퇴 후 재입학을 원할 경우, 재입학 절차를 따르게 됩니다. 
> 
> > 자퇴에 대해서 주의할 점은 자퇴 후에는 학적이 말소되어 학업을 이어나갈 수 없으며, 재입학을 원할 경우에는 별도의 절차를 거쳐야 한다는 것입니다. 
> 
> #### 수강에 대한 답변 
> > **계절수업**은 정규 학기 외에 여름방학과 겨울방학 동안 진행되는 수업을 말합니다. 계절수업은 학점 취득을 위해 개설되며, 학생들은 이를 통해 학점을 추가로 취득할 수 있습니다. 
> 
> > 계절수업은 정규 학기와 동일한 방식으로 진행되며, 수강신청, 강의평가, 시험 및 성적 평가 등이 포함됩니다. 계절수업을 통해 학점을 취득하면 졸업 요건을 충족하는 데 도움이 됩니다. 
> 
> > 계절수업에 대해서 주의할 점은 수강신청 기간과 수업 일정이 정규 학기와 다르므로, 이를 미리 확인하고 준비해야 한다는 것입니다. 또한, 계절수업은 추가 비용이 발생할 수 있습니다. 
> 
> ### 추천 행동
> |행동 | 이유 | 설명 |
> |---|---| ---|
> |자퇴 신청서 제출 | 자퇴를 원할 경우 | 자퇴를 원할 경우, 학사관리팀에 자퇴 신청서를 제출해야 합니다. |
> |계절수업 수강신청 | 학점 추가 취득 | 계절수업을 통해 학점을 추가로 취득할 수 있으며, 졸업 요건을 충족하는 데 도움이 됩니다. |
> |학사관리팀 상담 | 정확한 정보 확인 | 자퇴나 계절수업에 대한 자세한 정보를 얻기 위해 학사관리팀과 상담하는 것이 좋습니다. |
> 
> ### 추가정보
> [강남대학교 학사안내](https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do)

In [25]:
# 컬렉션의 모든 항목 가져오기
results = collection.get()

# 데이터를 출력하여 확인
for doc in results['documents']:
    print(f"Document ID: {doc['id']}")
    print(f"Embedding: {doc['embedding']}")
    print(f"Metadata: {doc['metadata']}")
    print("-------------------------------")
